# Importing the libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score,precision_score,recall_score,f1_score,mean_absolute_error

## importing the dataset

In [3]:
df = pd.read_csv("new_data.csv")

In [4]:
df.head()

,Unnamed: 0,spend,clicks,impressions,sales,orders,ROAS,bid,price,id,date
0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.016817e+10,2022-07-24
1,1,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,6.016817e+10,2022-07-29
2,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.016817e+10,2022-07-30
3,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.016817e+10,2022-08-01
4,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.016817e+10,2022-08-03


In [5]:
# checking the datatypes
df.dtypes

Unnamed: 0       int64
spend          float64
clicks         float64
impressions    float64
sales          float64
orders         float64
ROAS           float64
bid            float64
price          float64
id             float64
date            object
dtype: object

In [6]:
# Changing the dates column to datetime datatype and dropping the unnecessary column

df.drop(columns = "Unnamed: 0",inplace = True)
df["Dates"] = pd.to_datetime(df.date,format = "%Y-%m-%d")
df.drop(columns = "date",inplace = True)
df["day"] = df["Dates"].dt.day
df["month"] = df["Dates"].dt.month
df.drop(columns = "id",inplace = True)
df.drop(columns = "Dates",inplace = True)
df["day"]   = df["day"].astype("float64")
df["month"] = df["month"].astype("float64")

In [7]:
# Checking the datatypes
df.dtypes

spend          float64
clicks         float64
impressions    float64
sales          float64
orders         float64
ROAS           float64
bid            float64
price          float64
day            float64
month          float64
dtype: object

#### Dividing the data into the X and Y datasets

In [8]:
X = df[['spend', 'clicks', 'impressions', 'sales', 'orders', 'ROAS','price', 'day', 'month']]
Y = df[["bid"]]

In [9]:
final_dict = {"Feature":[],"Score":[],"Model_name":[],"rmse_val":[],"r2_score":[],"mse":[],"mae":[]}
def add_record(df,feature,model,rmse,r2,mse,mae):
    df['Feature'].append(feature)
    df["Score"].append(score)
    df["Model_name"].append(model)
    df['rmse_val'].append(rmse)
    df["r2_score"].append(r2)
    df["mse"].append(mse)
    df['mae'].append(mae)

### Feature Selection

In [10]:
X = df[["spend","clicks", "impressions", "sales", "orders","ROAS","price","day","month"]]
Y = df[["bid"]]
new_y = Y[['bid']]  
a = ["clicks","impressions","orders","price","day","month"]

In [11]:
for i in a:

    new_x = X[['spend','sales','ROAS',i,"day","month"]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    # Standarizing the dataset
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(y_train)
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(y_test)
    
    # Model training 
    model = ElasticNet(alpha = 0.1)
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    add_record(final_dict,i,"ElasticNet Regression",rmse,r2,mse,mae)

In [12]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae
0,clicks,"[0.38214760190344355, 0.3837017838807153]",ElasticNet Regression,0.787778,0.383702,0.620594,0.515842
1,impressions,"[0.38214760190344355, 0.3837017838807153]",ElasticNet Regression,0.787778,0.383702,0.620594,0.515842
2,orders,"[0.38214760190344355, 0.3837017838807153]",ElasticNet Regression,0.787778,0.383702,0.620594,0.515842
3,price,"[0.38214760190344355, 0.3837017838807153]",ElasticNet Regression,0.787778,0.383702,0.620594,0.515842
4,day,"[0.38214760190344355, 0.3837017838807153]",ElasticNet Regression,0.787778,0.383702,0.620594,0.515842
5,month,"[0.38214760190344355, 0.3837017838807153]",ElasticNet Regression,0.787778,0.383702,0.620594,0.515842


## Building a predictive system

In [13]:
def predict(model):
    new_x = X[['spend','sales','clicks','ROAS',"day","month"]]
    new_y = Y[["bid"]]

    # Perform the train test split

    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)

    # Standarizing the dataset

    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(y_train)
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(y_test)

    # Model training 
    model = model()
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))


    spend = float(input("Enter the spend value: "))
    sales = float(input("Enter the sales value: "))
    ROAS = float(input("Enter the ROAS value: "))
    clicks = float(input("Enter the clicks value: "))
    day = float(input("Enter the day value: "))
    month = float(input("Enter the month value: "))
    
    print("The Expected bid value is : ",model.predict([[spend,sales,clicks,ROAS,day,month]]))

#### Call the function

In [14]:
predict(ElasticNet)

Enter the spend value: 23
Enter the sales value: 03
Enter the ROAS value: 031
Enter the clicks value: 2
Enter the day value: 9
Enter the month value: 2
The Expected bid value is :  [1.85633622]


# Project Completed by - Swetanshu Pandey